# <연애 상담 챗봇(Custom_data)>

## 1. 모델/데이터 불러오기 

In [2]:
# !pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# SBERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 학습된 input, output df 불러오기
data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v2(custom_data)/data/csv/연애시뮬레이션(실험)_전처리.csv')
data = data[['input','output']]

# input별 embedding data 불러오기
embedding_data = torch.load('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v2(custom_data)/data/embedding_data/custom_love_embedding_data.pt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f18cfcb428cd878ffa5d020e779acca6daa6e67c9f3240fecea49a31f541daee
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


## 2. 챗봇 대화 함수 선언

In [3]:
last_sentence = ''
last_sim_idx_list = []
overlap_cnt = 0

def talk_to_bot(sentence):

  global last_sentence 
  global last_sim_idx_list
  global overlap_cnt

  overlap_thresholds = 0.85
  remove_sim_idx = None

  sentence = sentence.replace(" ","")
  sentence_encode = model.encode(sentence)
  sentence_tensor = torch.tensor(sentence_encode)

  last_sentence = last_sentence.replace(" ","")
  last_sentence_encode = model.encode(last_sentence)
  last_sentence_tensor = torch.tensor(last_sentence_encode)

  overlap_cos_sim = util.cos_sim(sentence_tensor, last_sentence_tensor)

  cos_sim = util.cos_sim(sentence_tensor, embedding_data)

  # 이전 질문과 문장이 0.85이상 유사하면 
  # 해당 질문-답변 출력 X
  # 총 3번까지

  if overlap_cos_sim >= overlap_thresholds and overlap_cnt < 3 :
    print('************************ 이전 질문과 유사함 ************************')
    remove_sim_idx_list = last_sim_idx_list

    for i in remove_sim_idx_list:
      cos_sim[0][int(i)] = 0
    overlap_cnt += 1
    print(f'이전 출력 idx: {last_sim_idx_list}')
    print(f'이전 질문과 유사도: {overlap_cos_sim}')
    print(f'중복횟수 : {overlap_cnt}')
  else:
    overlap_cnt = 0
    last_sim_idx_list = []

  print(f'사용자의 질문: {sentence} \n')

  dot_score = util.dot_score(sentence_tensor, embedding_data)
  ascending_cos_sim = np.sort(cos_sim)
  descending_cos_sim = ascending_cos_sim[::-1]
  # print(f'cos_sim 유사도 내림차순: {ascending_cos_sim}')

  print(f'--------------------------- 최우선 순위 질문/답변 ---------------------------')
  print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

  # data에서 선택된 질문 출력
  best_sim_idx = int(np.argmax(cos_sim))
  sentence_qes = data['input'][best_sim_idx]
  print(f"선택된 질문 = {sentence_qes}")

  # data에서 선택된 질문 문장에 대한 인코딩
  selected_qes_encode = model.encode(sentence_qes)

  # 유사도 점수 측정
  # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
  # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
  # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
  print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
  score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
  print(f"직접 수식 작성한 코사인 유사도 = {score}")

  # 답변
  answer = data['output'][best_sim_idx]
  print(f"\n답변 : {answer}\n")

  # 중복 input 대응을 위해 이전 idx,sentence 저장 
  last_sim_idx_list.append(int(np.argmax(cos_sim)))
  last_sentence = sentence

  # 최우선 순위 질문 벡터 초기화 
  cos_sim[0][best_sim_idx] = 0

  print(f'--------------------------- 후순위 답변/질문 디버깅 ---------------------------')
  for i in range(1,6):
    print(f'<{i}순위 Case>')
    print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최우선 순위 질문 벡터 초기화 
    cos_sim[0][best_sim_idx] = 0

  print(f'-------------------------------------------------------------------------------')

  return last_sentence, last_sim_idx_list , overlap_cnt

## 3. 챗봇 실행 
## ex) talk_to_bot('ㅎㅇ')

In [4]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('ㅎㅇ')

사용자의 질문: ㅎㅇ 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 354
선택된 질문 = ㅎㅇ
util.cos_sim 활용 코사인 유사도 : 1.0
직접 수식 작성한 코사인 유사도 = 0.9999999403953552

답변 : 안녕~ 나는 연애상담 봇이야! 연애 고민이 있다면 내가 상담해줄게!

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 367
선택된 질문 = ㅃㅇ
util.cos_sim 활용 코사인 유사도 : 0.6493110656738281
직접 수식 작성한 코사인 유사도 = 1.079608678817749

답변 : 내가 도움이 됐으면 좋겠다! 힘내~ 나는 언제나 네 옆에 있어! 나중에 또 얘기하러 와 줘 ♥

<2순위 Case>
가장 높은 코사인 유사도 idx : 503
선택된 질문 = 그럼
util.cos_sim 활용 코사인 유사도 : 0.42562997341156006
직접 수식 작성한 코사인 유사도 = 1.021207332611084

답변 : 그럼 난 널 응원해

<3순위 Case>
가장 높은 코사인 유사도 idx : 352
선택된 질문 = 하이
util.cos_sim 활용 코사인 유사도 : 0.3485909700393677
직접 수식 작성한 코사인 유사도 = 0.9966026544570923

답변 : 안녕~ 나는 연애상담 봇이야! 연애 고민이 있다면 내가 상담해줄게!

<4순위 Case>
가장 높은 코사인 유사도 idx : 366
선택된 질문 = 빠이
util.cos_sim 활용 코사인 유사도 : 0.3262704610824585
직접 수식 작성한 코사인 유사도 = 1.0061665773391724

답변 : 내가 도움이 됐으면 좋겠다! 힘내~ 나는 언제나 네 옆에 있어! 나중에 또 얘기하

In [5]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('ㅎㅇ')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [354]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 1
사용자의 질문: ㅎㅇ 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 367
선택된 질문 = ㅃㅇ
util.cos_sim 활용 코사인 유사도 : 0.6493110656738281
직접 수식 작성한 코사인 유사도 = 1.079608678817749

답변 : 내가 도움이 됐으면 좋겠다! 힘내~ 나는 언제나 네 옆에 있어! 나중에 또 얘기하러 와 줘 ♥

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 503
선택된 질문 = 그럼
util.cos_sim 활용 코사인 유사도 : 0.42562997341156006
직접 수식 작성한 코사인 유사도 = 1.021207332611084

답변 : 그럼 난 널 응원해

<2순위 Case>
가장 높은 코사인 유사도 idx : 352
선택된 질문 = 하이
util.cos_sim 활용 코사인 유사도 : 0.3485909700393677
직접 수식 작성한 코사인 유사도 = 0.9966026544570923

답변 : 안녕~ 나는 연애상담 봇이야! 연애 고민이 있다면 내가 상담해줄게!

<3순위 Case>
가장 높은 코사인 유사도 idx : 366
선택된 질문 = 빠이
util.cos_sim 활용 코사인 유사도 : 0.3262704610824585
직접 수식 작성한 코사인 유사도 = 1.0061665773391724

답변 : 내가 도움이 됐으면 좋겠다! 힘내~ 나는 언제나 네 옆에 있어! 나중에 또 얘기하러 와 줘 ♥

<4순위 Case>
가장 높은 코사인 유사도 idx : 358
선택된